# Importing Necessary Libraries

In [2]:
#Importing libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import re

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv("/content/drive/MyDrive/spam_detection/processed_data.csv", encoding = 'utf-8')
df.head()

,label,subject,email_to,email_from,message
0,1,"Generic Cialis, branded quality@",the00@speedy.uwaterloo.ca,"""Tomas Jacobs"" <RickyAmes@aol.com>",Content-Type: text/html;\nContent-Transfer-Enc...
1,0,Typo in /debian/README,debian-mirrors@lists.debian.org,Yan Morin <yan.morin@savoirfairelinux.com>,"Hi, i've just updated from the gulus and I che..."
2,1,authentic viagra,<the00@plg.uwaterloo.ca>,"""Sheila Crenshaw"" <7stocknews@tractionmarketin...","Content-Type: text/plain;\n\tcharset=""iso-8859..."
3,1,Nice talking with ya,opt4@speedy.uwaterloo.ca,"""Stormy Dempsey"" <vqucsmdfgvsg@ruraltek.com>","Hey Billy, \n\nit was really fun going out the..."
4,1,or trembling; stomach cramps; trouble in sleep...,ktwarwic@speedy.uwaterloo.ca,"""Christi T. Jernigan"" <dcube@totalink.net>",Content-Type: multipart/alternative;\n ...


In [5]:
df.tail()

,label,subject,email_to,email_from,message
75414,1,the reply for your request for a job place [le...,"""Gnitpick"" <gnitpick@flax9.uwaterloo.ca>","""Sydney Car Centre"" <Merrill8783@168city.com>",Content-Type: text/html;\nContent-Transfer-Enc...
75415,1,the reply for your request for a job place [le...,"""Gnitpick"" <gnitpick@flax9.uwaterloo.ca>","""Sydney Car Centre"" <Merrill8783@168city.com>",Content-Type: text/html;\nContent-Transfer-Enc...
75416,0,"Re: [R] Me again, about the horrible documenta...",Duncan Murdoch <murdoch@stats.uwo.ca>,Philippe Grosjean <phgrosjean@sciviews.org>,"For those who are interested, I just cook a li..."
75417,0,Re: [R] RODBC problem,<r-help@stat.math.ethz.ch>,=?iso-8859-1?Q?Bernhard_Wellh=F6fer?=\n\t<Bern...,"Hello,\n\nas I wrote I call\n\n sqlFetch(chan..."
75418,1,I wanted the desk at his own laws: of the. Bu...,the00@plg.uwaterloo.ca,"""Danny"" <pwcusnt@noblecoffee.com>",Content-Type: multipart/alternative;\n\tbounda...


In [6]:
for i in range(10):
    print("Message: " + df.loc[i]['message'])
    print("-----------------------------------------------------------------------------------")

Message: Content-Type: text/html;
Content-Transfer-Encoding: 7Bit Do you feel the pressure to perform and not rising to the occasion?? Try V ia gr a ..... your anxiety will be a thing of the past and you will be back to your old self.
-----------------------------------------------------------------------------------
Message: Hi, i've just updated from the gulus and I check on other mirrors.
It seems there is a little typo in /debian/README file

Example:
http://gulus.usherbrooke.ca/debian/README
ftp://ftp.fr.debian.org/debian/README

"Testing, or lenny.  Access this release through dists/testing.  The
current tested development snapshot is named etch.  Packages which
have been tested in unstable and passed automated tests propogate to
this release."

etch should be replace by lenny like in the README.html



-- 
Yan Morin
Consultant en logiciel libre
yan.morin@savoirfairelinux.com
514-994-1556


-- 
To UNSUBSCRIBE, email to debian-mirrors-REQUEST@lists.debian.org
with a subject of "un

# EDA

In [7]:
# Checking for missing values
df.isnull().sum()

label            0
subject        793
email_to       576
email_from       0
message       1487
dtype: int64

In [8]:
# Dropping null values
df.dropna(inplace=True)
print(df.isnull().sum())

label         0
subject       0
email_to      0
email_from    0
message       0
dtype: int64


In [9]:
df.groupby("label").count()

,subject,email_to,email_from,message
label,,,,
0,25192,25192,25192,25192
1,47606,47606,47606,47606


In [10]:
# Renaming columns
df.rename(columns={'subject': 'sub', 'email_to': 'to', 'email_from': 'from', 'message': 'msg'}, inplace=True)
sd = df
df.head()

,label,sub,to,from,msg
0,1,"Generic Cialis, branded quality@",the00@speedy.uwaterloo.ca,"""Tomas Jacobs"" <RickyAmes@aol.com>",Content-Type: text/html;\nContent-Transfer-Enc...
1,0,Typo in /debian/README,debian-mirrors@lists.debian.org,Yan Morin <yan.morin@savoirfairelinux.com>,"Hi, i've just updated from the gulus and I che..."
2,1,authentic viagra,<the00@plg.uwaterloo.ca>,"""Sheila Crenshaw"" <7stocknews@tractionmarketin...","Content-Type: text/plain;\n\tcharset=""iso-8859..."
3,1,Nice talking with ya,opt4@speedy.uwaterloo.ca,"""Stormy Dempsey"" <vqucsmdfgvsg@ruraltek.com>","Hey Billy, \n\nit was really fun going out the..."
4,1,or trembling; stomach cramps; trouble in sleep...,ktwarwic@speedy.uwaterloo.ca,"""Christi T. Jernigan"" <dcube@totalink.net>",Content-Type: multipart/alternative;\n ...


In [11]:
print(str(round(25192/47606,2))+'%')

0.53%


In [12]:
# Counting the labels
df_spam = sd[sd['label'] == 1]
df_ham = sd[sd['label'] == 0]
print("Ham Dataset Shape:", df_ham.shape)
print("Spam Dataset Shape:", df_spam.shape)

Ham Dataset Shape: (25192, 5)
Spam Dataset Shape: (47606, 5)


In [13]:
# Decreasing the 'Spam' labels by 60%
target_spam_samples = int(df_spam.shape[0] * 0.4) 
df_spam_downsampled = df_spam.sample(target_spam_samples)
df_spam_downsampled.shape

(19042, 5)

In [14]:
# Combining the updated dataframe
df_balanced = pd.concat([df_ham , df_spam_downsampled])
df_balanced['label'].value_counts()

0    25192
1    19042
Name: label, dtype: int64

In [15]:
# Dropping to Column
df_balanced.drop(['to'], axis = 1, inplace = True)   
df_balanced.head()

,label,sub,from,msg
1,0,Typo in /debian/README,Yan Morin <yan.morin@savoirfairelinux.com>,"Hi, i've just updated from the gulus and I che..."
9,0,[R] Confidence-Intervals.... help...,"""Jochen.F"" <jjfahr@ucalgary.ca>",Hi...\n\nI have to use R to find out the 90% c...
19,0,Re: [R] Confidence-Intervals.... help...,"""Sarah Goslee"" <sarah.goslee@gmail.com>",Hm... sounds like a homework problem to me...\...
20,0,Re: Broken ACPI with kernel 2.6.18-4-686 in HP...,Gerhard Brauer <gerhard.brauer@web.de>,Gruesse!\n* V.Harishankar schrieb am [08.04.07...
28,0,[R] Failure of mcsamp() but not mcmcsamp(),Michael Kubovy <kubovy@virginia.edu>,"Daer r-helpers,\n\nCan anyone help with the fo..."


In [16]:
#Lower casing the strings
df_balanced['msg'] = df_balanced['msg'].str.lower()
df_balanced['sub'] = df_balanced['sub'].str.lower()
df_balanced['from'] = df_balanced['from'].str.lower()
df_balanced.head()

,label,sub,from,msg
1,0,typo in /debian/readme,yan morin <yan.morin@savoirfairelinux.com>,"hi, i've just updated from the gulus and i che..."
9,0,[r] confidence-intervals.... help...,"""jochen.f"" <jjfahr@ucalgary.ca>",hi...\n\ni have to use r to find out the 90% c...
19,0,re: [r] confidence-intervals.... help...,"""sarah goslee"" <sarah.goslee@gmail.com>",hm... sounds like a homework problem to me...\...
20,0,re: broken acpi with kernel 2.6.18-4-686 in hp...,gerhard brauer <gerhard.brauer@web.de>,gruesse!\n* v.harishankar schrieb am [08.04.07...
28,0,[r] failure of mcsamp() but not mcmcsamp(),michael kubovy <kubovy@virginia.edu>,"daer r-helpers,\n\ncan anyone help with the fo..."


In [17]:
#Removing punctuations and numbers from 'msg' column
df_balanced['msg'] = df_balanced['msg'].apply(lambda x:re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", '', x))
df_balanced['msg'] = df_balanced['msg'].apply(lambda x: re.sub(r'\b\w{10,}\b', '', x))
df_balanced.msg = df_balanced.msg.replace(r'\s+', ' ', regex=True)
df_balanced['msg'] = df_balanced['msg'].str.replace('textplain', '')
df_balanced.head()

,label,sub,from,msg
1,0,typo in /debian/readme,yan morin <yan.morin@savoirfairelinux.com>,hi ive just updated from the gulus and i check...
9,0,[r] confidence-intervals.... help...,"""jochen.f"" <jjfahr@ucalgary.ca>",hii have to use r to find out the 90 for the o...
19,0,re: [r] confidence-intervals.... help...,"""sarah goslee"" <sarah.goslee@gmail.com>",hm sounds like a homework problem to memaybe s...
20,0,re: broken acpi with kernel 2.6.18-4-686 in hp...,gerhard brauer <gerhard.brauer@web.de>,gruesse schrieb am 080407 0649 i have a proble...
28,0,[r] failure of mcsamp() but not mcmcsamp(),michael kubovy <kubovy@virginia.edu>,daer anyone help with the get chain monte carl...


In [18]:
# Importing 
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [19]:
# Preprocessing the data before training the model
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def rmv_stopwords(string):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(string)
    filtered_tokens = [w for w in tokens if not w in stop_words]
    filtered_sentence = ' '.join(filtered_tokens)
    return filtered_sentence


lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    words = text.split()
    words = [lemmatizer.lemmatize(word,pos='v') for word in words]
    return ' '.join(words)

df_balanced['msg'] = df_balanced['msg'].apply(rmv_stopwords)
df_balanced['msg'] = df_balanced['msg'].apply(lemmatize_words)
df_balanced[df_balanced['msg'].map(lambda x: x.isascii())]

df_balanced.head()

,label,sub,from,msg
1,0,typo in /debian/readme,yan morin <yan.morin@savoirfairelinux.com>,hi ive update gulus check mirrorsit seem littl...
9,0,[r] confidence-intervals.... help...,"""jochen.f"" <jjfahr@ucalgary.ca>",hii use r find 90 follow testa test multiple s...
19,0,re: [r] confidence-intervals.... help...,"""sarah goslee"" <sarah.goslee@gmail.com>",hm sound like homework problem memaybe start f...
20,0,re: broken acpi with kernel 2.6.18-4-686 in hp...,gerhard brauer <gerhard.brauer@web.de>,gruesse schrieb 080407 0649 problem hp compaq ...
28,0,[r] failure of mcsamp() but not mcmcsamp(),michael kubovy <kubovy@virginia.edu>,daer anyone help get chain monte carlo mcmc 1e...


# Model Building

In [20]:
# Storing the columns in varaibles
string = df_balanced['msg']
labels = df_balanced['label']

# Splitting the data for training
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(string, labels, test_size=0.3, random_state=58)

In [21]:
# Function for measuring performance of a model
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, PrecisionRecallDisplay, RocCurveDisplay
def perform(y_pred):
    print("Precision : ", precision_score(y_test, y_pred))
    print("Recall : ", recall_score(y_test, y_pred))
    print("Accuracy Score : ", accuracy_score(y_test, y_pred))
    print("F1 Score : ", f1_score(y_test, y_pred))
    print("\n", confusion_matrix(y_test, y_pred))
    print("")

Naive Bayes

In [30]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

# Initializing and fit the TF-IDF vectorizer
vectorization = TfidfVectorizer()
X_train_vectorized = vectorization.fit_transform(X_train)

# Saving the TF-IDF vectorizer to a file
joblib.dump(vectorization, 'tfidf_vectorizer.pkl')

# Transforming the test data using the fitted vectorizer
X_test_vectorized = vectorization.transform(X_test)

# Training the Naive Bayes classifier
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_vectorized, y_train)

# Saving the trained Naive Bayes classifier to a file
joblib.dump(naive_bayes, 'spam_predictor.pkl')

# Loading the saved models
vectorizer = joblib.load('tfidf_vectorizer.pkl')
model = joblib.load('spam_predictor.pkl')

# Using the loaded models for prediction
X_test_vectorized = vectorizer.transform(X_test)
y_pred = model.predict(X_test_vectorized)

perform(y_pred)

Precision :  0.9897979203453011
Recall :  0.8849324679880722
Accuracy Score :  0.9466505915153341
F1 Score :  0.9344323022781995

 [[7518   52]
 [ 656 5045]]



# Testing

Custom List

In [23]:
spam_email_texts = [
    "Dear Friend, Are you tired of working long hours without seeing any significant results? Our revolutionary money-making system can change your life. Join our exclusive program today and start earning thousands of dollars within weeks. Limited spots available, so act now! Regards, The Wealth Revolution Team",
    "Dear Lucky Winner, Congratulations on being selected as the winner of a luxurious cruise vacation. Pack your bags and prepare for an unforgettable journey. To claim your prize, simply provide your contact details. Don't miss out on this incredible opportunity! Best regards, The Cruise Adventures Team",
    "Dear Customer, We regret to inform you that your account is at risk of being suspended. To prevent this, please click the link below and update your account information immediately. Failure to do so may result in permanent account termination. Take action now! Sincerely, The Account Services Department",
    "Hello, For a limited time, we're offering a massive 80% discount on our exclusive collection of designer watches. Treat yourself or someone special to a luxurious timepiece at unbeatable prices. Don't wait - shop now before stocks run out! Best regards, The Watch Boutique",
    "Dear Job Seeker, Are you tired of searching for a job without success? We have the solution you've been waiting for. Gain access to our vast database of high-paying job opportunities and take your career to new heights. Register today and land your dream job! Regards, The Career Success Team",
    "Dear Sir/Madam, I am reaching out to you with a confidential business proposal of great financial potential. If you are interested in making substantial profits through a mutually beneficial partnership, please respond to this email. Your prompt action is appreciated. Best regards, Mr. Johnson Brown",
    "Hello, Struggling with a low credit score? We have the solution to your financial worries. Our credit repair program guarantees a rapid boost to your credit score. Say goodbye to high-interest rates and denied loan applications. Take control of your financial future today! Sincerely, The Credit Repair Experts",
    "Dear Valued Customer, Your bank account requires immediate verification. To ensure the security of your funds, please click the link below and provide your account details. Failure to do so may result in the suspension of your banking services. Protect your money now! Thank you, The Banking Security Team",
    "Dear Investor, You have been granted VIP access to our exclusive network of secret investment opportunities. Multiply your wealth by investing in high-profit ventures not available to the general public. Seize this chance to secure your financial future. Join our elite circle today! Regards, The Investment Mastery Group",
    "Hello, Looking to lose weight fast? Our groundbreaking weight loss formula guarantees rapid results. Say goodbye to diets and exercise routines that don't work. Try our miracle product and witness the pounds melt away effortlessly. Get the body you've always dreamed of! Best regards, The Slim & Fit Team",
    "Dear Email User, Your inbox is nearing its storage limit, causing potential disruptions in receiving new emails. Upgrade to our unlimited email storage plan today and never miss an important message again. Don't let a full inbox hold you back - expand your storage now! Sincerely, The Email Solutions Team~",
    "Dear Lucky Winner, Congratulations! You have won a substantial sum of money in our international lottery. To claim your prize, provide your contact details and a small processing fee. Act quickly to ensure your winnings are delivered without delay. Celebrate your newfound wealth! Best regards, The Lottery Commission",
    "Dear Taxpayer, Good news! You are eligible for a substantial tax refund. To expedite the process, click the link below and provide your bank account details. Don't miss out on this opportunity to receive your refund promptly. Act now! Regards, The Internal Revenue Service",
    "Hello, Rejuvenate your skin with our revolutionary anti-aging cream. Sign up for a free trial and experience the incredible transformation firsthand. Bid farewell to wrinkles and fine lines - embrace a youthful appearance today! Sincerely, The Ageless Beauty Team",
    "Dear Job Seeker, Escape the daily grind and work from the comfort of your home. Our proven work-from-home system guarantees a steady income stream. Join our network of successful remote workers and start living life on your terms. Take the first step toward financial freedom now! Best regards, The Work-From-Home Success Team",
    "Dear Beneficiary, I am contacting you regarding an unclaimed inheritance that you are entitled to. Your cooperation is needed to facilitate the release of the funds. Please respond to this email to initiate the necessary proceedings. Time is of the essence - don't let this opportunity slip away! Regards, Barrister James Johnson",
    "Hello, Escape to paradise with our discounted vacation packages. Experience luxurious accommodations, breathtaking landscapes, and unforgettable adventures. Don't let this amazing deal pass you by book your dream vacation today! Best regards, The Travel Deals Team",
    "Dear Website Owner, Boost your online presence and drive massive traffic to your website with our guaranteed SEO services. Dominate search engine rankings and watch your business thrive. Take the first step toward online success today! Sincerely, The SEO Masters",
    "Dear Valued Customer, Congratulations! You have been pre-approved for a generous personal loan with low interest rates. Secure the funds you need for your dreams, whether it's a home renovation, education expenses, or debt consolidation. Don't let financial constraints hold you back! Thank you, The Loan Approval Team",
    "Dear Friend, I am Prince John, seeking your urgent assistance to transfer a large sum of money out of my country. Your cooperation is vital, and you will be generously rewarded for your help. Please reply to this email to discuss further. Confidentiality is of utmost importance. Best regards, Prince John",
    "Dear Beneficiary, We have discovered unclaimed funds in your name, awaiting immediate release. Act now to secure your rightful inheritance or unclaimed funds. Provide your personal information and pay a small processing fee to initiate the transfer. Don't miss out on this financial windfall! Regards, The Fund Release Department",
    "Dear Entrepreneur, Imagine earning a staggering $10,000 per week from the comfort of your home. Our proven system guarantees financial success without any special skills or experience. Join our exclusive program and unlock the path to wealth and freedom. Start living the life you deserve! Best regards, The Financial Freedom Team",
    "Hello, Upgrade your style with our exquisite collection of luxury watches at an unbeatable discount of 90%. From renowned brands to timeless classics, we have it all. Don't miss this opportunity to adorn your wrist with elegance and sophistication. Shop now! Best regards, The Watch Emporium",
    "Dear PayPal User, We regret to inform you that your PayPal account has been temporarily limited due to suspicious activity. To restore full access, click the link below and provide your account details. Failure to do so may result in permanent account suspension. Secure your account now! Sincerely, The PayPal Security Team",
    "Dear Lucky Winner, Congratulations on being selected as the winner of a brand-new iPhone X. Claim your prize by providing your contact details and paying a small shipping fee. Don't miss out on this amazing opportunity to own the latest smartphone. Act fast! Regards, The Prize Claim Department",
    "Dear Investor, I am reaching out to you with an urgent business proposal that guarantees high returns on your investment. Take advantage of this exclusive opportunity and maximize your wealth. Respond to this email to discuss further. Secure your financial future now! Best regards, Mr. Smith Anderson",
    "Hello, Achieve your weight loss goals with our revolutionary supplements. For a limited time, enjoy a massive discount on our proven formulas. Say goodbye to excess weight and hello to a healthier, fitter you. Don't wait - start your weight loss journey today! Sincerely, The Slim & Trim Team",
    "Dear Valued Customer, Your bank account requires an immediate update to comply with new security measures. Click the link below and provide your account information to avoid service interruption. Protect your funds from unauthorized access. Act now! Thank you, The Bank Security Team",
    "Dear Trader, Unlock the secrets of successful Forex trading with our exclusive invitation to join the elite circle. Learn from industry experts, receive insider tips, and maximize your profits. Don't miss out on this opportunity to become a Forex trading master. Join us now! Regards, The Forex Mastery Group",
    "Dear Job Seeker, Tired of your 9-to-5 job? Earn an incredible $1,000 per day with our proven work-from-home program. No special skills or experience required. Start living life on your terms and achieve financial freedom. Take the first step towards a better future today! Best regards, The Work-From-Home Success Team",
    "Dear Friend, I am contacting you with a confidential business proposal that offers immense financial rewards. Your assistance is crucial in transferring a substantial sum of money. This opportunity is time-sensitive, so respond promptly for further instructions. Trust and discretion are of utmost importance. Best regards, Dr. Williams Brown",
    "Hello, Struggling with a low credit score? Our credit repair experts guarantee significant improvement in your credit rating. Say goodbye to financial roadblocks and unlock a world of opportunities. Take control of your financial future today! Sincerely, The Credit Repair Solutions",
    "Hello, Escape to paradise with our discounted vacation packages to exotic destinations. Experience breathtaking landscapes, luxurious accommodations, and unforgettable adventures. Don't let this incredible deal slip away - book your dream getaway now! Best regards, The Travel Deals Team",
    "Dear Website Owner, Is your website struggling to attract visitors? Our professional SEO services guarantee increased traffic, higher search engine rankings, and improved online visibility. Take your online presence to new heights and watch your business thrive! Sincerely, The SEO Experts",
    "Dear Taxpayer, We have noticed that your tax payment is overdue. Failure to settle the outstanding amount immediately may result in penalties and legal consequences. Click the link below to make a secure payment and avoid further complications. Act now! Regards, The Internal Revenue Service",
    "Hello, Rejuvenate your skin with our breakthrough anti-wrinkle cream. Sign up for a free trial and experience visible results in just days. Embrace a youthful appearance and regain your confidence. Try it now! Sincerely, The Beauty Revitalization Team",
    "Dear Entrepreneur, Ditch the traditional office and work from anywhere while earning big. Our remote work program offers flexibility, independence, and unlimited earning potential. Embrace the digital lifestyle and take control of your future. Join us today! Best regards, The Remote Work Revolution",
    "Dear Business Owner, I am contacting you with an exclusive business proposal that promises lucrative returns. Join forces with us and dominate the market in a mutually beneficial partnership. Respond to this email to discuss further. Don't miss out on this extraordinary opportunity! Best regards, Mr. Johnson Smith",
    "Hello, Get fit and save big with our limited-time offer on gym memberships. Enjoy state-of-the-art facilities, expert trainers, and a motivating environment. Don't miss this chance to prioritize your health and well-being. Join us today! Best regards, The Fitness Center",
    "Dear Valued Customer, Congratulations! You have been selected as a winner in our customer appreciation sweepstakes. Claim your prize by providing your contact details and a small processing fee. Don't miss out on this fantastic reward! Regards, The Sweepstakes Committee",
    "Dear Investor, Our cutting-edge investment platform offers unparalleled opportunities for wealth creation. Join our network of successful investors and unlock the potential for substantial profits. Secure your financial future today! Regards, The Investment Opportunities Team",
    "Hello, Upgrade your style with our exclusive collection of designer handbags. From iconic classics to the latest trends, we have the perfect bag for every occasion. Indulge in luxury and elevate your fashion game. Shop now! Best regards, The Designer Boutique",
    "Dear PayPal User, Urgent action required! We have detected unauthorized access to your PayPal account. To protect your funds, click the link below and verify your account information. Failure to do so may result in permanent account suspension. Secure your account now! Sincerely, The PayPal Security Team",
    "Dear Lucky Winner, Congratulations on winning a dream vacation for two. Claim your prize by providing your contact details and paying a small processing fee. Don't miss out on this incredible opportunity to explore the world. Act now! Regards, The Prize Claim Department",
    "Dear Investor, I am reaching out to you with an exclusive investment opportunity that guarantees high returns and financial security. Respond to this email to learn more about this limited-time offer. Your financial future starts now! Best regards, Mr. David Thompson",
    "Hello, Lose weight effortlessly with our breakthrough weight loss program. Shed pounds without dieting or exercise and achieve your ideal body. Say goodbye to extra weight and hello to a healthier, happier you. Get started today! Sincerely, The Weight Loss Experts"
]

non_spam_email_texts = [
  "Dear Haaland, We are delighted to invite you to the Annual Conference on Environmental Sustainability, taking place on 24th of Feb at Trest. This prestigious event brings together industry leaders, researchers, and policymakers to discuss key challenges and innovations in the field of environmental sustainability. We believe your expertise and insights would greatly contribute to the discussions. Please find attached the conference agenda and registration details. We kindly request you to confirm your attendance by 29th of Jan. We look forward to your participation and the opportunity to exchange ideas with you. Best regards, Solomon Ramsey Gates Foundation",
  "Dear Werghoust, We hope this email finds you well. We are writing to confirm that we have received your payment for Invoice #123456. The amount of $2000 has been successfully credited to your account. We appreciate your promptness in settling the invoice and your continued support. If you have any questions or require further information, please feel free to reach out to our accounts department. Thank you for your business, and we look forward to serving you in the future. Warm regards, Anthony DallE",
  "Dear Sebastian, Thank you for applying for the Python Developer position at Google. We acknowledge receipt of your application materials and appreciate the time and effort you put into the application process. Our hiring team will carefully review your application to determine the next steps. We will be in touch within 6 hours to inform you of the status of your application. In the meantime, please feel free to reach out if you have any questions or need further information. We appreciate your interest in joining our team and wish you the best of luck in your job search. Sincerely, Sarah Google",
  "Dear Hamilton, We are pleased to extend our invitation to you for the Annual Charity Gala, benefiting Melinda Organization. This highly anticipated event will be held on 27th of August, 2024 at Sacre Cou'er. The evening promises an exquisite dining experience, live entertainment, and a silent auction featuring unique items. Your support plays a vital role in enabling us to continue our charitable endeavors. Attached to this email, you will find the event details and ticket purchase information. We kindly request your presence at this special occasion, as your participation will make a difference in the lives of those in need. If you have any questions or require further information, please do not hesitate to contact our event coordinator. We eagerly anticipate your response and look forward to welcoming you at the gala. Warm regards, Mike Pound Melinda Charity",
  "Dear Abdullah, I hope this email finds you well. I wanted to provide you with an update on the status of the ML Model. Our team has made significant progress since our last communication. Key milestones achieved include: Completed Phase 1 of the project ahead of schedule. Successfully conducted user testing and incorporated valuable feedback. Finalized the project timeline for Phase 2. We are confident that the project is progressing according to plan and will be delivered on time. Should you have any questions or require further information, please feel free to reach out to me or the project team. Thank you for your continued support and collaboration. We look forward to sharing further updates as we move forward. Best regards, Ahmad FAANG",
  "Dear Reyna, At WallMart, we value your feedback and strive to continuously improve our products and services. To help us better understand your experience and preferences, we kindly invite you to participate in our annual Customer Satisfaction Survey. Your opinion is important to us, and the survey will only take approximately 3 minutes to complete. Your responses will remain anonymous and will be used solely for research purposes. We genuinely appreciate your time and input, as it will assist us in enhancing our offerings to better meet your needs. To access the survey, please click on the following link: https://bitly.co/2p3jfi234. The survey will be available until 28th of February, 2023. We thank you in advance for your participation and look forward to serving you better in the future. Sincerely, Brimstone WallMart",
  "Dear Chamber, This is a friendly reminder to save the date for the upcoming event gala on 29th of March, 2023 at Jobs Opera House. We are excited to host this event, which promises to be an informative and engaging experience. The agenda includes keynote speeches from renowned industry experts, panel discussions, and networking opportunities. Further details regarding the event schedule and registration will be shared in the coming weeks. We highly encourage you to mark your calendar and join us for this enriching event. If you have any questions or need additional information, please do not hesitate to reach out to our event coordinator. We look forward to welcoming you at the Event Gala. Best regards, Breach K Productions",
  "Dear Feroz, I hope this email finds you well. My name is Saima Zafar, and I am a HOD at FAST-NUCES. I have been following your work on ECG Monitoring System with great interest, and I believe there could be significant synergies between our research endeavors. I am reaching out to explore the possibility of collaboration on a research project that focuses on AI Acceleration. Given your expertise and contributions to the field, I believe your involvement would greatly enhance the project's outcomes. I would be honored to discuss this opportunity further and provide more details about the project. If you are interested, please let me know your availability for a meeting or a call at your convenience. I look forward to the possibility of working together. Thank you for your consideration. Best regards, Saima Zafar HOD FAST-NUCES, Lahore",
  "Dear Marven, We are thrilled to celebrate your work anniversary at Meta. Time flies, and it's hard to believe that 5 years have already passed since you joined our team. We wanted to take a moment to express our heartfelt appreciation for your dedication, hard work, and contributions. Your commitment to excellence has not gone unnoticed, and your positive impact on the organization and your colleagues is commendable. We value your expertise and the unique perspectives you bring to the table. Thank you for being an invaluable part of our team. Please accept our sincere congratulations on reaching this milestone. We are excited to see what the future holds for you and are grateful for your continued presence at Meta. Warm regards, Jacob Meta",
  "Dear all, We would like to inform you about an upcoming company-wide town hall meeting scheduled for 23/09/2023 at 7:00 PM. This event will be an excellent opportunity for all employees to gather and receive updates on the latest company news, achievements, and future plans. During the town hall meeting, our CEO, Darren, will address the entire organization and provide insights into our strategic direction and goals. Additionally, the meeting will include an open Q&A session where you can ask questions and share your thoughts. We highly encourage your attendance as your presence and participation are vital to the success of this event. Please mark your calendar and make the necessary arrangements to join us on 23/09/2023 at the auditorium. Further details, including the agenda, will be shared closer to the date. Thank you for your attention, and we look forward to seeing you at the town hall meeting. Sincerely, HR Team Haas F1 Team",
  "Dear Abdullah, We are thrilled to invite you to the upcoming industry conference, 'Innovations in Technology,' taking place on March 9, 2023, at Hudderfield. This event brings together thought leaders, experts, and innovators in the technology sector to explore the latest trends and advancements shaping our industry. As a respected professional in the field, we believe your presence and insights would greatly enrich the conference. The agenda includes keynote speeches, panel discussions, and networking opportunities with industry peers. We are confident that this event will inspire new ideas and foster valuable connections. Please find attached the conference schedule and registration details. We kindly request you to confirm your attendance by February 12, 2023. Should you have any questions or require further information, please do not hesitate to contact our event coordinator. We look forward to welcoming you to the conference and the opportunity to engage in meaningful discussions. Best regards, Alex Kersten Linus Media Group",
  "Dear Hamilton, We would like to remind you about the upcoming webinar on 'Mastering Social Media Marketing' hosted by Reddit. This highly informative session will provide valuable insights into the strategies and techniques for leveraging social media platforms to drive business growth. Date: 23rd of June 2024 Time: 1:00 PM PST Duration: 3 hours To join the webinar, simply click on the following link: https://www.zoom.com/32f2j?02if. We recommend joining a few minutes before the scheduled start time to ensure a seamless experience. Please note that the webinar will be recorded and made available for future reference. If you have any questions or need technical assistance, our support team is readily available to assist you. We hope you find this webinar valuable and look forward to your participation. Best regards, Nicholos Friedman",
  "Dear Mr. Pound, Thank you for choosing Nvidia as your preferred provider of GPU Accelerations. We strive to deliver the highest quality and ensure our customers' satisfaction. To help us better understand your experience and identify areas for improvement, we kindly request your feedback through our Customer Satisfaction Survey. Your opinions are invaluable to us, and the survey will only take a few minutes to complete. Your responses will remain anonymous and will be used to enhance our products and services. As a token of our appreciation, you will have the opportunity to enter a draw to win $2000 upon completing the survey. To access the survey, please click on the following link: https://bitly.co/48530923j2. We encourage you to provide honest and detailed feedback to help us serve you better. Your input is instrumental in shaping our future endeavors. Thank you in advance for your time and participation. We look forward to continuing to exceed your expectations. Sincerely, Mike Nvidia Inc.",
  "Dear all, We hope this email finds you well. We would like to inform you about the implementation of new company policies and procedures aimed at improving efficiency, enhancing communication, and maintaining a positive work environment. These updates have been carefully crafted to align with our company's values and support our shared goals. To familiarize yourself with the new policies, please review the attached document outlining the changes and guidelines. We encourage you to ask questions and seek clarification from your supervisors or the HR department if needed. It is crucial that we all adhere to these policies to ensure a harmonious and productive workplace. We understand that change can be challenging, but we firmly believe that these updates will contribute to our collective success. Your cooperation and compliance are greatly appreciated. Thank you for your attention and ongoing commitment to Microsoft's growth. Best regards, Management Team Microsoft Inc.",
  "Dear Hammond, We are delighted to extend our invitation to the exclusive product launch event of the RTX 5000 series GPUs. This highly anticipated event will take place on 12th of May, 2024, at the CES'24. Join us as we unveil the cutting-edge features and demonstrate the remarkable capabilities of our latest innovation. The event will feature live product demonstrations, guest speakers, and networking opportunities with industry experts. As a valued customer, your presence is of utmost importance to us. We would be honored to have you witness firsthand how our new GPUs can transform your business. Please find attached the event details and RSVP information. Due to limited availability, we kindly request you to confirm your attendance by 21st of December, 2023. Should you have any questions or require further information, please do not hesitate to contact our event coordinator. We look forward to celebrating this milestone with you and showcasing the future of GPUs. Best regards, Mariana Nvidia Corp.",
  "Dear Abdullah, We hope this email finds you well. We are writing to inform you about a project deadline extension for Spam Email Detection. After careful consideration and evaluation of various factors, we have decided to extend the deadline to ensure the successful completion of the project without compromising its quality. The revised deadline for the project is now 4th of August, 2023. This extension will allow our team to address unforeseen challenges and incorporate additional feedback and improvements. We believe that this adjustment will result in a more robust and impactful final deliverable. We understand that this may impact your schedule, and we appreciate your flexibility and understanding. Should you have any concerns or require further clarification, please do not hesitate to reach out to us. We remain committed to delivering an exceptional outcome and exceeding your expectations. Thank you for your cooperation and continued support throughout this project. Best regards, Nigel Toro Rosso",
  "Dear Ali, We are excited to invite you to volunteer at the upcoming Donation Drive, a community initiative aimed at making a positive impact on our local neighborhood. This event will take place on 14th of May, 2023, at LDA Model Bazaar, and we believe your involvement can contribute to its success. As a volunteer, you will have the opportunity to engage with community members, participate in various activities, and support meaningful causes. Whether it's assisting with event setup, coordinating workshops, or providing general assistance, your time and effort will make a difference. Attached to this email, you will find further details about the event and the available volunteer roles. We kindly request you to review the information and indicate your preferred role(s). Feel free to reach out to our volunteer coordinator if you have any questions or require additional information. Thank you for considering this invitation, and we look forward to working together to create a positive impact in our community. Best regards, Sajid Akhuwat Foundation",
  "Dear Altair, We hope this email finds you well and that you continue to enjoy the benefits of Wifi Services. At StormFiber, we take great pride in providing exceptional experiences to our valued customers. We would like to request your assistance in sharing your experience with the 5G WiFi Servies by providing us with a testimonial. Your testimonial will help potential customers understand the value and quality we strive to deliver. Please feel free to add any additional comments or feedback you may have. We greatly appreciate your time and contribution to our ongoing success. With your permission, we may include your testimonial on our website, social media platforms, and marketing materials. If you prefer not to have your name associated with the testimonial, please let us know, and we will ensure anonymity. Thank you for your support, and we look forward to receiving your valuable feedback. Best regards, Saeed StormFiber",
  "Dear William, We are conducting a research study on DRS Performance Increase and would like to invite you to participate. This study aims to gather insights and opinions from individuals with expertise in the Motorsport Industry to contribute to the advancement of knowledge in this area. Your unique perspective and experience make you an ideal candidate for this study. Your participation will involve the Tire Degradation Study of Slick Tyres. We assure you that all information provided will be kept confidential and used only for research purposes. If you are interested in participating, please reply to this email, and our research team will provide you with further details and answer any questions you may have. Your contribution will be greatly appreciated and acknowledged in the final research report. Thank you for considering this invitation, and we look forward to your participation. Sincerely, Redbull Racing Team",
  "Dear Sergio, We hope this email finds you well. We would like to inform you that the open enrollment period for company benefits is approaching. This is an opportunity for you to review and make any necessary changes to your current benefits elections. During this period, you will have the chance to explore the various benefit options available and make informed decisions based on your individual needs and preferences. Attached to this email, you will find the updated benefits package information, including details on health insurance plans, retirement contributions, flexible spending accounts, and other available options. We encourage you to carefully review this information and consult with our HR department or benefits administrator if you have any questions or require assistance. Please note that any changes made during the open enrollment period will take effect on Friday. Failure to make any changes will result in your current benefits elections rolling over into the new plan year. We value the well-being of our employees and believe that our comprehensive benefits package plays a crucial role in supporting your overall satisfaction and work-life balance. We are committed to providing you with the resources and support you need to thrive both personally and professionally. Thank you for your attention, and we appreciate your active participation in the open enrollment process. Best regards, HR Team Ford America",
  "Hey Mohsin, Long time no talk! It feels like ages since we last caught up. How have you been? Let's plan a get-together soon and catch up on all the latest happenings in our lives. I'd love to hear how things have been going for you. Are you free this weekend? Looking forward to seeing you and sharing some good laughs! Take care, Momin",
  "Dear Meisam, I hope this email finds you in good health and high spirits! I wanted to let you know that we are planning a family reunion this summer. It's been too long since we all gathered under one roof, and I thought it would be a great opportunity for us to reconnect, reminisce, and create new memories together. We are in the process of finalizing the details, including the date, venue, and activities. I'll keep you updated as we progress. In the meantime, please let me know if you have any preferences or suggestions. We want to make this reunion a special and memorable event for everyone. Looking forward to seeing you and the whole family soon! Warm regards, Abdullah",
  "Hi Musa, I hope you're having a productive day at work! I could use a little break from my tasks, and I thought it would be nice to catch up over a cup of coffee. Are you up for a quick coffee break in the afternoon? Let's chat about non-work-related stuff and unwind for a bit. Let me know if you're available, and we can pick a time that works for both of us. Cheers, Taha",
  "Hey Talha, The weekend is just around the corner, and I wanted to see if you're up for some fun and relaxation. I heard about this new movie that everyone's raving about, and I thought it would be great to watch it together. What do you think? We can grab dinner beforehand and make it a mini movie night! Let me know if you're interested and available. If not, we can brainstorm some other ideas and make the most of our time off. Take care, Zulfiqar",
  "Dear Usman, Happy, happy birthday to you! 🎉🎂🥳 I hope this email brings you lots of joy and warmth on your special day. May this year be filled with love, laughter, and countless memorable moments. You deserve nothing but the best! Let's plan a celebration soon. I can't wait to shower you with gifts and celebrate the amazing person you are. Wishing you a fantastic year ahead! Sending hugs and good vibes your way, Saad",
  "Hi Danish, I couldn't wait to send you this email to congratulate you on your well-deserved promotion! 🎉🥳 You've worked incredibly hard, and it's inspiring to see your dedication pay off. This promotion is a testament to your skills, knowledge, and commitment to excellence. I'm truly thrilled for you and can't wait to see all the amazing things you'll accomplish in your new role. If there's anything I can do to support you during this transition, please don't hesitate to reach out. Congratulations once again, and keep shining! Best regards, Kaleb",
  "Hey Anne, I hope you're doing well! I recently tried out this delicious recipe, and it turned out to be a hit. I thought of you because I know how much you enjoy cooking and trying new dishes. I'd love to share the recipe with you and hear your thoughts on it. If you're interested, let me know, and I'll send you the recipe right away. And if you have any exciting recipes to share, I'm all ears! Let's exchange some culinary inspiration. Take care, Gordon",
  "Hi Michael, It's been ages since we had a proper catch-up session, and I miss your company. How about we meet up for lunch next week? We can try that new restaurant everyone's been talking about and spend some quality time together. Let me know which day works best for you, and I'll make the reservation. I can't wait to hear all about your latest escapades and share some stories of my own. Looking forward to it! Warm regards, Chloe",
  "Dear Alex, I wanted to take a moment to express my heartfelt gratitude for your assistance with my FYP. Your support and expertise were instrumental in its successful completion. I couldn't have done it without you! Your willingness to go the extra mile and lend a helping hand is truly commendable. Your professionalism and teamwork are an inspiration to everyone on our team. I'm grateful to have you as a colleague and a friend. Thank you again, and please don't hesitate to reach out if there's ever anything I can do for you. Best regards, Abdullah",
  "Hey Saad, Since we can't meet up in person right now, I thought it would be fun to organize a virtual game night. We can gather our friends online and play some of our favorite games together. It'll be a great way to bond, have some laughs, and beat the boredom. Let me know if you're interested, and I'll start planning the details. I'm open to suggestions on which games to play, so feel free to share your ideas. Looking forward to a night of friendly competition! Take care, Abdullah",
  "Hi Abdullah, I hope this email finds you well! As discussed during our last family gathering, we're forming a reunion planning committee to ensure a memorable and smooth reunion for everyone. Your participation would be highly valued and appreciated. The committee will be responsible for deciding on the date, location, activities, and other logistics. We want to make this reunion a truly special event that caters to everyone's preferences. If you're interested in joining the committee, please let me know, and I'll add you to the group. Let's work together to create a reunion that will bring us all closer and leave us with cherished memories. Warm regards, Admin",
  "Hey Kyle, I hope you're doing well. I wanted to take a moment to thank you for the amazing birthday surprise you organized for me. I was completely blown away by your thoughtfulness and the effort you put into making my day so special. From the decorations to the delicious cake and the heartwarming messages from our friends, everything was perfect. It truly made me feel loved and appreciated. Thank you once again for being such an incredible friend. I'm grateful to have you in my life. Sending you big hugs, Nigel",
  "Hi, I hope you're having a productive day at work! I wanted to extend an invitation for lunch this week. It's been a while since we had a chance to chat outside of the office, and I thought it would be great to catch up. There's a new café nearby that I've been wanting to try. How about we give it a go on Wednesday? It would be a nice break from our usual routine and a chance to relax and enjoy each other's company. Let me know if you're available, and we can finalize the details. Take care, Mason",
  "Dear Abdullah, I wanted to take a moment to congratulate you on your remarkable achievement - graduating with flying colors! 🎓🎉 Your hard work, dedication, and perseverance have paid off, and I couldn't be prouder of you. This graduation marks the beginning of a new chapter in your life, filled with endless possibilities and exciting opportunities. I have no doubt that you will excel in whatever path you choose. Take a moment to savor this accomplishment, and remember that your family is always here to support and cheer you on. Congratulations once again, and here's to a bright future! With love, Mom",
  "Hey, I hope this email finds you well and in high spirits. It's been too long since we had a good old-fashioned dinner party, and I thought it's time to change that. I'm planning a cozy dinner party at my place on Friday. The menu will consist of some of our favorite dishes, and of course, there will be plenty of laughter and good conversation. Please let me know if you're available and if you have any dietary preferences or restrictions. I want to make sure everyone feels comfortable and enjoys the evening to the fullest. Looking forward to having you over! Warm regards, Max",
  "Dear Mom, Happy Mother's Day to the best dad in the world! Today is a special day to celebrate and honor you for all the love, guidance, and support you've given me throughout my life. I'm grateful every day for the incredible father you are. Thank you for being my role model, my pillar of strength, and my biggest cheerleader. Your wisdom, kindness, and sense of humor have shaped me into the person I am today. I hope you have a fantastic day filled with love, joy, and relaxation. You deserve all the happiness in the world. Sending you all my love",
  "Congratz, I heard the exciting news and couldn't be happier for you - congratulations on your new home! 🏡🎉 This is a significant milestone in your life, and I know how much this means to you. Wishing you endless happiness and wonderful memories in your new abode. May your new home be a place of warmth, comfort, and love. If there's anything I can do to help during the moving process, please don't hesitate to reach out. I'm here to lend a hand and celebrate this exciting chapter with you. Congratulations once again! Warmest regards, Keaton",
  "Hi, I hope you're having a great day at work! I could use a little break from the hustle, and I thought it would be nice to grab a coffee and catch up. It's been a while since we had a chance to chat about things outside of work. Let me know if you're up for a coffee break this week, and we can find a convenient time that works for both of us. Looking forward to catching up! Best regards, Nicole"
]

In [24]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [25]:
# Function to remove stopwords from text
def rmv_stopwords(string):
    tokens = word_tokenize(string)
    filtered_tokens = [w for w in tokens if not w in stop_words]
    filtered_sentence = ' '.join(filtered_tokens)
    return filtered_sentence

# Function to Lemmatize the words in a text
def lemmatize_words(text):
    words = text.split()
    words = [lemmatizer.lemmatize(word, pos='v') for word in words]
    return ' '.join(words)

# Function to Preprocess the dataframe
def pre_proc(email_text):
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", '', email_text)
    text = re.sub(r'\b\w{10,}\b', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.replace('textplain', '')
    return text

In [26]:
# Testing for Spam Emails
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

correct = 0
incorrect = 0
incorrect_predictions = []

for text in spam_email_texts:
    clean_text = pre_proc(text)
    processed_text = lemmatize_words(rmv_stopwords(text))
    test_string_vectorized = vectorization.transform([processed_text])
    prediction = naive_bayes.predict(test_string_vectorized)
    
    if prediction == 1:
        correct += 1
    else:
        incorrect += 1
        incorrect_predictions.append(text)

print("Total number of data: ", len(spam_email_texts))
print("Correct predictions:", correct)
print("Incorrect predictions:", incorrect)
print("Percentage: ", (correct/len(spam_email_texts))*100)
print(" ")
print("Incorrect prediction texts:")
for text in incorrect_predictions:
    print(text)
    print(" ")

Total number of data:  46
Correct predictions: 36
Incorrect predictions: 10
Percentage:  78.26086956521739
 
Incorrect prediction texts:
Dear Job Seeker, Are you tired of searching for a job without success? We have the solution you've been waiting for. Gain access to our vast database of high-paying job opportunities and take your career to new heights. Register today and land your dream job! Regards, The Career Success Team
 
Dear Email User, Your inbox is nearing its storage limit, causing potential disruptions in receiving new emails. Upgrade to our unlimited email storage plan today and never miss an important message again. Don't let a full inbox hold you back - expand your storage now! Sincerely, The Email Solutions Team~
 
Dear Taxpayer, Good news! You are eligible for a substantial tax refund. To expedite the process, click the link below and provide your bank account details. Don't miss out on this opportunity to receive your refund promptly. Act now! Regards, The Internal Re

In [27]:
# Testing for legitimate emails

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

correct = 0
incorrect = 0
incorrect_predictions = []

for text in non_spam_email_texts:
    clean_text = pre_proc(text)
    processed_text = lemmatize_words(rmv_stopwords(clean_text))
    test_string_vectorized = vectorization.transform([processed_text])
    prediction = naive_bayes.predict(test_string_vectorized)

    
    if prediction == 0:
        correct += 1
    else:
        incorrect += 1
        incorrect_predictions.append(text)

print("Total number of data: ", len(non_spam_email_texts))
print("Correct predictions:", correct)
print("Incorrect predictions:", incorrect)
print("Percentage: ", (correct/len(non_spam_email_texts))*100)
print(" ")
print("Incorrect prediction texts:")
for text in incorrect_predictions:
    print(text)
    print(" ")

Total number of data:  38
Correct predictions: 31
Incorrect predictions: 7
Percentage:  81.57894736842105
 
Incorrect prediction texts:
Dear Usman, Happy, happy birthday to you! 🎉🎂🥳 I hope this email brings you lots of joy and warmth on your special day. May this year be filled with love, laughter, and countless memorable moments. You deserve nothing but the best! Let's plan a celebration soon. I can't wait to shower you with gifts and celebrate the amazing person you are. Wishing you a fantastic year ahead! Sending hugs and good vibes your way, Saad
 
Hi Danish, I couldn't wait to send you this email to congratulate you on your well-deserved promotion! 🎉🥳 You've worked incredibly hard, and it's inspiring to see your dedication pay off. This promotion is a testament to your skills, knowledge, and commitment to excellence. I'm truly thrilled for you and can't wait to see all the amazing things you'll accomplish in your new role. If there's anything I can do to support you during this tr